In [ ]:
from swagger_parser import SwaggerParser

parser = SwaggerParser(swagger_path='swagger.json')  # Init with file

# Get an example of dict for the definition Foo
parser.definitions_example.get('Pet')

In [1]:
import json

with open("swagger.json","r") as infile:
    swagger = json.load(infile)

In [2]:
from faker import Faker
from chance import chance
import random
fake = Faker()



In [74]:
property_generators = {
    "name":lambda x: fake.name(),
    "date-time": lambda x: fake.date_time(),
    "username":lambda x: chance.email(),
    "email": lambda x: chance.email(),
    "id":lambda x: fake.uuid4(),
    "quantity":lambda x: random.randint(0,100),
    "integer":lambda x: random.randint(0,100),
    "bool": lambda x: chance.boolean(),
    "boolean": lambda x: chance.boolean(),
    "firstName": lambda x: fake.name().split(" ")[0],
    "lastName": lambda x: fake.name().split(" ")[-1],
    "password": lambda x: fake.password(),
    "phone": lambda x: fake.phone_number(),
    "url": lambda x: chance.url(),
    "enum": lambda x: random.choice(x[0]["enum"]),
    "array": lambda x: [x[1] + str(i) for i in range(random.randint(1,10))],
    "$ref": lambda x: list(generate(x[0]["$ref"].split("/")[-1])),
}

object_generators = {}

def generate_from_property_key(k, prop,prop_name):
    #print("--" + k)
    if k in property_generators:
        return property_generators[k]((prop,prop_name))
    elif prop[k] in property_generators:
        return property_generators[prop[k]]((prop,prop_name))
    
           
def generate_property_value(prop, prop_name):    
    if prop_name in property_generators:
        val = property_generators[prop_name]((prop,prop_name))
        if val is not None:
            #print("---" + str(val))
            return val
    
    # iterate through object keys for the property
    for key in prop.keys():
        val = generate_from_property_key(key, prop, prop_name)
        if val is not None:
            #print("---" + str(val))
            return val
        #else:
            #print("---NONE")
    raise Exception(prop)

def get_generator_for_definition(definition_name):
    definition = swagger['definitions'][definition_name] 
    def object_generator():
        instance = {}
        
        # iterate through properties 
        for prop_name in definition['properties']:
            #print("-" + prop_name)
            prop = definition['properties'][prop_name]
            try:
                generated_prop = generate_property_value(prop, prop_name)
            except Exception as e:
                print(prop_name)
                raise e
            instance[prop_name] = generated_prop
        yield instance
    return object_generator
    
for definition_name in swagger["definitions"]:
    object_generators[definition_name] = get_generator_for_definition(definition_name)

def generate(obj_name):
    print(obj_name)
    if obj_name not in object_generators:
        raise Exception("Object " + obj_name + " is unknown")
    return object_generators[obj_name]()

In [72]:
list(generate("Order"))

Order


[{'id': '235c9c2e-3fe3-4889-9acb-c082305f277d',
  'petId': 44,
  'quantity': 77,
  'shipDate': datetime.datetime(1996, 6, 24, 21, 14, 39),
  'status': 'delivered',
  'complete': True}]

In [102]:
summaries = []

for path_name in swagger["paths"]:
    path = swagger["paths"][path_name]
    for method in path:
        command = path[method]["summary"] if len(path[method]["summary"]) > 0 else path[method]["description"]
        for param in path[method]["parameters"]:
            if param["in"] == "header":
                continue
           
            #if param["in"] == "path" or param["in"] == "query" or param["in"] == "body" or param["in"] == "formData":
            if "schema" in param:
                if "$ref" in param["schema"]:
                    obj = list(generate(param["schema"]["$ref"].split("/")[-1]))[0]
                elif param["schema"]["type"] == "array":
                    obj = generate_from_property_key("array", param, param["name"])
            elif param["type"] == "array":
                if "enum" in param:
                    obj = generate_from_property_key("enum", param["items"], param["name"])
                else:
                    obj = generate_from_property_key("array", param, param["name"])
            else:
                obj = generate_from_property_key("name", param, param["name"])

        summaries.append((command, obj))        
summaries

Pet
Category
Pet
Category
Order
User
User


[('Add a new pet to the store',
  {'id': '67088548-8ffd-4f47-b4ea-1a77ecdf70e7',
   'category': [{'id': '35aaf600-abcd-492f-a670-e9f034a6c43b',
     'name': 'Robert Howell'}],
   'name': 'Debbie Burke',
   'photoUrls': ['photoUrls0',
    'photoUrls1',
    'photoUrls2',
    'photoUrls3',
    'photoUrls4',
    'photoUrls5',
    'photoUrls6'],
   'tags': ['tags0', 'tags1', 'tags2', 'tags3'],
   'status': 'pending'}),
 ('Update an existing pet',
  {'id': '19371716-903a-430e-96bb-71aad717178a',
   'category': [{'id': '34e1f7e8-1246-438e-a8a9-0c6a9f7cb9dc',
     'name': 'Richard Ramos'}],
   'name': 'Angela Henry',
   'photoUrls': ['photoUrls0'],
   'tags': ['tags0', 'tags1'],
   'status': 'sold'}),
 ('Finds Pets by status', ['status0', 'status1']),
 ('Finds Pets by tags', ['tags0', 'tags1']),
 ('Find pet by ID', 'Brandon Santos'),
 ('Updates a pet in the store with form data', 'Troy Herrera'),
 ('Deletes a pet', 'Theresa Walker'),
 ('uploads an image', 'Jessica Williams'),
 ('Returns pet in

['Add a new pet to the store',
 'Update an existing pet',
 'Finds Pets by status',
 'Finds Pets by tags',
 'Find pet by ID',
 'Updates a pet in the store with form data',
 'Deletes a pet',
 'uploads an image',
 'Returns pet inventories by status',
 'Place an order for a pet',
 'Find purchase order by ID',
 'Delete purchase order by ID',
 'Create user',
 'Creates list of users with given input array',
 'Creates list of users with given input array',
 'Logs user into the system',
 'Logs out current logged in user session',
 'Get user by user name',
 'Updated user',
 'Delete user']